In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from geopandas.tools import sjoin
from shapely.geometry import Point
from tqdm import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd

## Loading and Preparing the Traffic Data

In [2]:
# Let's load the main traffic dataset
traffic_df = pd.read_csv("Automated_Traffic_Volume_Counts_20250404.csv")

In [3]:
# Let's check the info about the dataset
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712605 entries, 0 to 1712604
Data columns (total 14 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   RequestID  int64 
 1   Boro       object
 2   Yr         int64 
 3   M          int64 
 4   D          int64 
 5   HH         int64 
 6   MM         int64 
 7   Vol        int64 
 8   SegmentID  int64 
 9   WktGeom    object
 10  street     object
 11  fromSt     object
 12  toSt       object
 13  Direction  object
dtypes: int64(8), object(6)
memory usage: 182.9+ MB


In [4]:
# And, check the head of the dataset
print(traffic_df.head())

   RequestID      Boro    Yr   M   D  HH  MM  Vol  SegmentID  \
0      32970    Queens  2021   4  30   2   0    0     149701   
1      32970    Queens  2021   4  30   2  15    1     149701   
2      11342  Brooklyn  2012  12  18   8  15   33      20063   
3      32970    Queens  2021   4  30   2  30    0     149701   
4      32970    Queens  2021   4  30   2  45    0     149701   

                                        WktGeom          street  \
0  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
1  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
2                     POINT (985746.5 167127.4)           61 ST   
3  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
4  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   

                    fromSt      toSt Direction  
0  Newtown Creek Shoreline  Dead end        NB  
1  Newtown Creek Shoreline  Dead end        NB  
2                    15 AV     16 AV        WB  
3  Newtown Creek

In [5]:
# Now, let's convert columns to appropriate types
traffic_df['Yr'] = traffic_df['Yr'].astype(int)
traffic_df['M'] = traffic_df['M'].astype(int)
traffic_df['D'] = traffic_df['D'].astype(int)
traffic_df['HH'] = traffic_df['HH'].astype(int)
traffic_df['MM'] = traffic_df['MM'].astype(int)
traffic_df['Vol'] = pd.to_numeric(traffic_df['Vol'], errors='coerce').fillna(0).astype(int)

# Let's combine Yr, M, D into a proper datetime object and create a date column
traffic_df['date'] = pd.to_datetime({
    'year': traffic_df['Yr'],
    'month': traffic_df['M'],
    'day': traffic_df['D']
})

### Hourly to Daily Traffic

In [6]:
# Let's aggregate volume to daily totals per segment
daily_traffic = traffic_df.groupby(['SegmentID', 'date']).agg({
    'Vol': 'sum',
    'WktGeom': 'first',
    'Boro': 'first',
    'street': 'first',
    'Direction': 'first'
}).reset_index()

# And, check the aggregated data
print(daily_traffic.head())

   SegmentID       date    Vol                                       WktGeom  \
0         44 2015-02-07  45506  POINT (914734.9916698899 120903.10647429322)   
1         44 2015-02-08  44512  POINT (914734.9916698899 120903.10647429322)   
2         44 2015-02-09  50696  POINT (914734.9916698899 120903.10647429322)   
3         44 2015-02-10  34657  POINT (914734.9916698899 120903.10647429322)   
4         44 2015-02-11  31097  POINT (914734.9916698899 120903.10647429322)   

            Boro       street Direction  
0  Staten Island  MOON AVENUE        SB  
1  Staten Island  MOON AVENUE        SB  
2  Staten Island  MOON AVENUE        NB  
3  Staten Island  MOON AVENUE        NB  
4  Staten Island  MOON AVENUE        SB  


### Geospatial Processing of the Traffic Data

In [7]:
# Let's convert WKTGeom to GeoDataFrame
daily_traffic['geometry'] = daily_traffic['WktGeom'].apply(wkt.loads)

# And, create GeoDataFrame (initial CRS: EPSG:2263, NY State Plane feet)
traffic_gdf = gpd.GeoDataFrame(daily_traffic, geometry='geometry', crs='EPSG:2263')

# And, convert to standard latitude/longitude CRS (EPSG:4326)
traffic_gdf = traffic_gdf.to_crs('EPSG:4326')

# And, also check the conversion
print(traffic_gdf.head())

   SegmentID       date    Vol                                       WktGeom  \
0         44 2015-02-07  45506  POINT (914734.9916698899 120903.10647429322)   
1         44 2015-02-08  44512  POINT (914734.9916698899 120903.10647429322)   
2         44 2015-02-09  50696  POINT (914734.9916698899 120903.10647429322)   
3         44 2015-02-10  34657  POINT (914734.9916698899 120903.10647429322)   
4         44 2015-02-11  31097  POINT (914734.9916698899 120903.10647429322)   

            Boro       street Direction                    geometry  
0  Staten Island  MOON AVENUE        SB  POINT (-74.24995 40.49825)  
1  Staten Island  MOON AVENUE        SB  POINT (-74.24995 40.49825)  
2  Staten Island  MOON AVENUE        NB  POINT (-74.24995 40.49825)  
3  Staten Island  MOON AVENUE        NB  POINT (-74.24995 40.49825)  
4  Staten Island  MOON AVENUE        SB  POINT (-74.24995 40.49825)  


In [8]:
# Now, let's drop unnecessary columns
traffic_gdf = traffic_gdf.drop(columns=['WktGeom'])

# And, save cleaned GeoDataFrame to file for spatial analysis
traffic_gdf.to_file('daily_traffic_newyork.geojson', driver='GeoJSON')

# Also save as CSV if needed
traffic_gdf.to_csv('daily_traffic_newyork.csv', index=False)

## Spatial Aggregation

In [2]:
# Let's load the bike data
bike_df = pd.read_csv('daily_cycling_data_newyork_07042025.csv')

# And, also the motor traffic data
motor_df = pd.read_csv('daily_traffic_newyork.csv')

In [3]:
# Convert dates to datetime format and extract year
bike_df['date'] = pd.to_datetime(bike_df['date'])
motor_df['date'] = pd.to_datetime(motor_df['date'])
motor_df['year'] = motor_df['date'].dt.year

In [4]:
# Unique dates in bike data
bike_dates = pd.to_datetime(bike_df['date']).dt.date.unique()
print("Number of unique dates in bike data:", len(bike_dates))

# Unique dates in motor data
motor_dates = pd.to_datetime(motor_df['date']).dt.date.unique()
print("Number of unique dates in motor data:", len(motor_dates))

# Find the intersection of dates
common_dates = set(bike_dates).intersection(set(motor_dates))
print("Number of common dates between bike and motor data:", len(common_dates))

# Dates missing in motor data
missing_dates = set(bike_dates) - set(motor_dates)
print("Number of bike data dates missing in motor data:", len(missing_dates))

# Print a few missing dates as a sample
print("Sample of missing dates:", list(missing_dates)[:10])

Number of unique dates in bike data: 3653
Number of unique dates in motor data: 4928
Number of common dates between bike and motor data: 2581
Number of bike data dates missing in motor data: 1072
Sample of missing dates: [datetime.date(2024, 4, 21), datetime.date(2020, 3, 26), datetime.date(2024, 4, 24), datetime.date(2018, 12, 29), datetime.date(2023, 12, 13), datetime.date(2021, 7, 23), datetime.date(2022, 1, 19), datetime.date(2022, 4, 19), datetime.date(2024, 9, 20), datetime.date(2020, 1, 3)]


In [5]:
# Now, let's create GeoDataFrames for bike data using EPSG:4326 and transforming to EPSG:2263
bike_gdf = gpd.GeoDataFrame(
    bike_df,
    geometry=gpd.points_from_xy(bike_df.longitude, bike_df.latitude),
    crs="EPSG:4326"
).to_crs(epsg=2263)

# And also, create GeoDataFrames for motor data using EPSG:4326 and transforming to EPSG:2263
motor_gdf = gpd.GeoDataFrame(
    motor_df,
    geometry=gpd.points_from_xy(motor_df['geometry'].apply(lambda x: float(x.split(' ')[1].strip('()'))),
                               motor_df['geometry'].apply(lambda x: float(x.split(' ')[2].strip('()')))),
    crs="EPSG:4326"
).to_crs(epsg=2263)

In [10]:
# Let's load the daily bike and motor traffic data
bike_df = pd.read_csv('daily_cycling_data_newyork_03042025.csv')
motor_gdf = gpd.read_file('daily_traffic_newyork.geojson')

# And, convert dates to datetime
bike_df['date'] = pd.to_datetime(bike_df['date'])
motor_gdf['date'] = pd.to_datetime(motor_gdf['date'])

In [11]:
# Now, let's convert bike data to GeoDataFrame
bike_gdf = gpd.GeoDataFrame(
    bike_df,
    geometry=gpd.points_from_xy(bike_df.longitude, bike_df.latitude),
    crs="EPSG:4326"
).to_crs(epsg=32618)

# And, also the motorized traffic data from the GeoJSON
motor_gdf = motor_gdf.to_crs(epsg=32618)

In [12]:
# Let's take out the unique dates from bicycle data
unique_dates = bike_gdf['date'].dt.date.unique()

In [13]:
# Prepare an empty results list
results = []

# Iterate through each unique date
for current_date in tqdm(unique_dates):
    # Filter daily bike and motor data
    bike_day = bike_gdf[bike_gdf['date'].dt.date == current_date].copy()
    motor_day = motor_gdf[motor_gdf['date'].dt.date == current_date].copy()

    # Skip if either dataset is empty for the current date
    if motor_day.empty or bike_day.empty:
        continue

    # Calculate city-wide average motorized volume for the day
    city_avg_motor_volume = motor_day['Vol'].mean()
    city_total_motor_vehicles = motor_day['Vol'].sum()

    # Loop through each bike counter on that day
    for idx, bike_row in bike_day.iterrows():
        # Create a 6 km buffer around the bike station
        buffer_geom = bike_row.geometry.buffer(6000)
        
        # Filter motorized traffic data within the buffer
        nearby_motor = motor_day[motor_day.within(buffer_geom)]

        # Skip if no motorized data is found within the buffer
        if nearby_motor.empty:
            continue

        # Aggregate nearby motor features
        avg_motor_volume = nearby_motor['Vol'].mean()
        total_motor_vehicles = nearby_motor['Vol'].sum()
        num_motor_sources = len(nearby_motor)

        # Append the result
        results.append({
            'date': current_date,
            'station_name': bike_row['name'],
            'latitude': bike_row['latitude'],
            'longitude': bike_row['longitude'],
            'bike_counts': bike_row['counts'],
            
            # Local (6 km radius) motor features
            'avg_motor_volume': avg_motor_volume,
            'total_motor_vehicles': total_motor_vehicles,
            'num_motor_sources': num_motor_sources,

            # City-wide motor features
            'city_avg_motor_volume': city_avg_motor_volume,
            'city_total_motor_vehicles': city_total_motor_vehicles
        })


100%|██████████████████████████████████████████████████████████████████████████████| 3653/3653 [04:15<00:00, 14.27it/s]


In [14]:
# Convert results to a DataFrame
final_df = pd.DataFrame(results)

# Save the merged data
final_df.to_csv('ny_bike_with_motor_traffic.csv', index=False)
print("Spatial aggregation completed and saved as 'ny_bike_with_motor_traffic.csv'")

Spatial aggregation completed and saved as 'ny_bike_with_motor_traffic.csv'


In [15]:
# Let's quickly check the merged file
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7750 entries, 0 to 7749
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       7750 non-null   object 
 1   station_name               7750 non-null   object 
 2   latitude                   7750 non-null   float64
 3   longitude                  7750 non-null   float64
 4   bike_counts                7750 non-null   int64  
 5   avg_motor_volume           7750 non-null   float64
 6   total_motor_vehicles       7750 non-null   int64  
 7   num_motor_sources          7750 non-null   int64  
 8   city_avg_motor_volume      7750 non-null   float64
 9   city_total_motor_vehicles  7750 non-null   int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 605.6+ KB


In [16]:
# Ensure the 'date' column is in datetime format
final_df['date'] = pd.to_datetime(final_df['date'])

# Extract the year from the date column
final_df['year'] = final_df['date'].dt.year

# Group by year and count unique dates
unique_days_per_year = final_df.groupby('year')['date'].nunique().reset_index()
unique_days_per_year.rename(columns={'date': 'unique_days'}, inplace=True)

# Display the result
print("Number of Unique Days for Each Year:")
print(unique_days_per_year)

Number of Unique Days for Each Year:
   year  unique_days
0  2015          177
1  2016          215
2  2017          217
3  2018          166
4  2019          222
5  2020           99
6  2021          163
7  2022          150
8  2023          197
9  2024           97


In [6]:
# Set the buffer size to 6 km (6000 meters)
buffer_size = 6000

# Precompute yearly median volume per segment
yearly_median_vol = motor_gdf.groupby(['SegmentID', 'year'])['Vol'].median().reset_index()

# Precompute station matches: find motor stations within 6 km of each bike station
station_matches = {}
for idx, bike_row in tqdm(bike_gdf.iterrows(), total=bike_gdf.shape[0], desc="Precomputing station matches"):
    buffer_geom = bike_row.geometry.buffer(buffer_size)
    nearby_motor = motor_gdf[motor_gdf.geometry.within(buffer_geom)]
    station_matches[bike_row['name']] = nearby_motor['SegmentID'].unique()

Precomputing station matches: 100%|██████████████████████████████████████████████| 36461/36461 [18:14<00:00, 33.32it/s]


In [7]:
# Initialize results list
results = []

# Process each date
for current_date in tqdm(bike_gdf['date'].dt.date.unique(), desc="Processing each date"):
    # Extract the current year
    current_year = pd.to_datetime(current_date).year
    
    # Filter bike and motor data for the current date
    bike_day = bike_gdf[bike_gdf['date'].dt.date == current_date].copy()
    motor_day = motor_gdf[motor_gdf['date'].dt.date == current_date].copy()

    # Calculate city-wide motor features
    if motor_day.empty:
        # Use the median of the specific year for city-wide calculations
        year_median_vol = yearly_median_vol[yearly_median_vol['year'] == current_year]['Vol'].median()
        city_total_motor_volume = year_median_vol * len(motor_gdf['SegmentID'].unique())
        city_avg_motor_volume = year_median_vol
    else:
        city_total_motor_volume = motor_day['Vol'].sum()
        city_avg_motor_volume = motor_day['Vol'].mean()

    # Loop through each bike station on that date
    for idx, bike_row in bike_day.iterrows():
        station_name = bike_row['name']
        nearby_segments = station_matches.get(station_name, [])

        # Filter motor data for the matched segments on the current date
        nearby_motor = motor_day[motor_day['SegmentID'].isin(nearby_segments)]

        # If no data for the date, use the median of that year for the specific segments
        if nearby_motor.empty:
            # Filter median for specific year and segments
            year_median_vol_segment = yearly_median_vol[
                (yearly_median_vol['SegmentID'].isin(nearby_segments)) & 
                (yearly_median_vol['year'] == current_year)
            ]['Vol'].median()
            
            # If the yearly median is also missing, use the global median as a fallback
            if pd.isna(year_median_vol_segment):
                year_median_vol_segment = motor_gdf['Vol'].median()

            # Create a placeholder with the computed median
            nearby_motor = pd.DataFrame([{
                'Vol': year_median_vol_segment,
                'SegmentID': ','.join(map(str, nearby_segments)),
                'geometry': bike_row.geometry
            }])

        # Aggregate nearby motor features
        results.append({
            'date': current_date,
            'station_name': bike_row['name'],
            'latitude': bike_row['latitude'],
            'longitude': bike_row['longitude'],
            'cycling_volume': bike_row['counts'],

            # Local (6 km radius) features
            'avg_motor_volume': nearby_motor['Vol'].mean(),
            'total_motor_volume': nearby_motor['Vol'].sum(),
            'num_motor_sources': len(nearby_motor),

            # City-wide features
            'city_avg_motor_volume': city_avg_motor_volume,
            'city_total_motor_volume': city_total_motor_volume
        })

Processing each date: 100%|████████████████████████████████████████████████████████| 3653/3653 [06:54<00:00,  8.82it/s]


In [8]:
# Convert results to DataFrame and save
final_df = pd.DataFrame(results)

# And, check the final_df
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36461 entries, 0 to 36460
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     36461 non-null  object 
 1   station_name             36461 non-null  object 
 2   latitude                 36461 non-null  float64
 3   longitude                36461 non-null  float64
 4   cycling_volume           36461 non-null  int64  
 5   avg_motor_volume         36461 non-null  float64
 6   total_motor_volume       36461 non-null  float64
 7   num_motor_sources        36461 non-null  int64  
 8   city_avg_motor_volume    36461 non-null  float64
 9   city_total_motor_volume  36461 non-null  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 2.8+ MB


In [9]:
final_df.head()

,date,station_name,latitude,longitude,cycling_volume,avg_motor_volume,total_motor_volume,num_motor_sources,city_avg_motor_volume,city_total_motor_volume
0,2022-05-09,111th St at 50th Ave,40.745630,-73.852500,120,17832.0,17832.0,1,15862.333333,47587.0
1,2022-05-09,Amsterdam Ave at 86th St.,40.787700,-73.975050,2053,5728.0,5728.0,1,15862.333333,47587.0
2,2022-05-09,Brooklyn Bridge Bicycle Path (Roadway),40.712656,-74.004464,3590,56957.0,56957.0,1,15862.333333,47587.0
3,2022-05-09,Brooklyn Bridge Bike Path,40.709274,-74.000990,26,56957.0,56957.0,1,15862.333333,47587.0
4,2022-05-09,Columbus Ave at 86th St.,40.787700,-73.975050,1561,5728.0,5728.0,1,15862.333333,47587.0


In [10]:
final_df.to_csv('bike_with_motor_traffic_nyc.csv', index=False)

print("Merged data saved as 'bike_with_motor_traffic_nyc.csv'")

Merged data saved as 'bike_with_motor_traffic_nyc.csv'


In [11]:
print(final_df.describe())
print("Number of unique dates:", final_df['date'].nunique())
print("Number of unique stations:", final_df['station_name'].nunique())

           latitude     longitude  cycling_volume  avg_motor_volume  \
count  36461.000000  36461.000000    36461.000000      36461.000000   
mean      40.721497    -73.973061     2151.641699      11171.669462   
std        0.047611      0.045053     2048.057501      13041.986564   
min       40.584100    -74.072075        0.000000          0.000000   
25%       40.709274    -73.994950      372.000000       5728.000000   
50%       40.714573    -73.971382     1725.000000       5917.000000   
75%       40.751010    -73.951492     3146.000000      13459.500000   
max       40.857669    -73.852500    13346.000000     154879.000000   

       total_motor_volume  num_motor_sources  city_avg_motor_volume  \
count        36461.000000       36461.000000           36461.000000   
mean         11302.867276           1.014838           10784.006405   
std          13306.458627           0.169780            9346.416826   
min              0.000000           1.000000               0.000000   
25%  